In [20]:
import ee
import geemap
import logging
import multiprocessing
import os
import requests
import shutil
from retry import retry

In [21]:
ee.Initialize(opt_url='https://earthengine.googleapis.com')

In [76]:
Map = geemap.Map()


In [77]:
StartDate= ee.Date("2021-01-01")
EndDate = ee.Date("2021-02-01")

In [78]:
region = ee.Geometry.Polygon([[-88.956, 17.073],[-86.243, 17.073], [-88.956, 15.671], [-86.243, 15.671]])
region2 = ee.Geometry.Polygon([[-86.243, 17.073],[-88.956, 17.073], [-88.956, 15.671], [-86.243, 15.671]], None, False)

In [79]:
# image = (ee.ImageCollection('COPERNICUS/S2_SR')
#          .filterDate(StartDate, EndDate)
#          .filterBounds(region2)
#          .mosaic()
#          .clip(region2)
#          .select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']))

In [80]:
image = ee.ImageCollection('COPERNICUS/S2_SR') \
            .filterBounds(region2) \
            .filterDate(StartDate, EndDate) \
            .select('B4', 'B3', 'B2') \
            .median() \
            .visualize(min=0, max=1000).clip(region2)

In [81]:
params = {
    'count': 1,  # How many image chips to export
    'buffer': 127,  # The buffer distance (m) around each point
    'scale': 10,  # The scale to do stratified sampling
    'seed': 1,  # A randomization seed to use for subsampling.
    'dimensions': '256x256',  # The dimension of each image chip
    'format': "GEO_TIFF",  # The output image format, can be png, jpg, ZIPPED_GEO_TIFF, GEO_TIFF, NPY
    'prefix': 'tile_',  # The filename prefix
    'processes': 25,  # How many processes to used for parallel processing
    'out_dir': './tiles/',  # The output directory. Default to the current working directly
}
# viz_params = {
#   'bands': ['B8', 'B4', 'B3'], 'min':0, 'max': 4000, 'gamma': 1
# }

In [85]:
Map.addLayer(image.sampleRegions(collection=ee.FeatureCollection(region2), scale=5000, geometries=True), {}, "points")
Map.addLayer(image, {}, "Image")
Map.addLayer(region2, {}, "ROI", False)
Map.setCenter(-88.956, 17.073)
Map

Map(bottom=863.0, center=[17.073, -88.956], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

In [ ]:
def getRequests():
    img = ee.Image(1).rename("Class").addBands(image)
    points = img.stratifiedSample(
        numPoints=params['count'],
        region=region2,
        scale=params['scale'],
        seed=params['seed'],
        geometries=True,
    )
    Map.data = points
    return points.aggregate_array('.geo').getInfo()

In [ ]:
@retry(tries=10, delay=1, backoff=2)
def getResult(index, point):
    print("running1..")
    point = ee.Geometry.Point(point)
    region = point.buffer(params['buffer']).bounds()
    print("running2..")
    if params['format'] in ['png', 'jpg']:
        url = image.getThumbURL(
            {
                'region': region,
                'dimensions': params['dimensions'],
                'format': params['format'],
            }
        )
    else:
        url = image.getDownloadURL(
            {
                'region': region,
                'dimensions': params['dimensions'],
                'format': params['format'],
            }
        )

    if params['format'] == "GEO_TIFF":
        ext = 'tif'
    else:
        ext = params['format']

    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()

    out_dir = os.path.abspath(params['out_dir'])
    basename = str(index).zfill(len(str(params['count'])))
    filename = f"{out_dir}{params['prefix']}{basename}.{ext}"
    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(r.raw, out_file)
    print("Done: ", basename)

In [ ]:
items = getRequests()
print(items)
num = 0
for i in items:
    index = num
    num+= 1
    point = i.get('coordinates')
    print(point)
    getResult(index, point)

In [12]:
Map.addLayer(Map.data, {}, "Sample points")
Map

Map(center=[17.073, -88.956], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…